In [ ]:
from spikeHelper.loadSpike import loadSpikeBehav,epochData
import pickle
import glob

binSize = 50
sigma= 50
for rat in [7,8,9,10]:
    filename='Data/r'+str(rat)+'_bin'+str(binSize)+'_sigma'+str(sigma)+'.pickle'
    
    if len(glob.glob(filename))==1:
        print('File',filename,'already exists. Loading filtered epochs')
        epochs = pickle.load(open(filename,'rb'))

    else:
        print('File',filename,'does not exist. Filtering epochs and saving file')
        spikes,behavior = loadSpikeBehav('Data/rato'+str(rat)+'tudo.mat', binSize=binSize)
        epochs = epochData(spikes,behavior,rat,sigma=sigma,binSize=binSize)
        pickle.dump(epochs,open(filename,'wb'))
    

File Data/r7_bin50_sigma50.pickle does not exist. Filtering epochs and saving file


---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from spikeHelper.visuals import trialNeuronPlot, firingRateEvo
from spikeHelper.loadSpike import loadSpikeBehav, filterEpochs
from spikeHelper.dataOrganization import XyTfromEpoch
from spikeHelper.filters import convHist, kernelSmooth,binarize
import pickle
import glob

%matplotlib inline

# Load Function

In [3]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

rat = 8

fileName = 'Data/rato'+str(rat)+'tudo.mat'
data = loadmat(fileName)

spikes = data['dados'][0,0][1]
behavior = data['dados'][0,0][0]

spikes = pd.DataFrame([[ spikes[0,i][0][:,0], spikes[0,i][0][:,1]] for i in range(spikes.shape[1]) if spikes[0,i][0].shape[1]==2], columns=['times','trial'])
behavior = pd.DataFrame(np.transpose(behavior[0,0][0]), columns=['one','onset','offset','zero','duration','sortIdx','sortLabel'])
spikes['trialTime'] = pd.DataFrame(np.transpose([spikes.times[i] - behavior.iloc[spikes.trial[i]-1].onset.as_matrix() for i in range(spikes.shape[0])]))

In [4]:
spikes.head()

times  \
0  [146.074296236, 146.147205114, 146.403532743, ...   
1  [147.650232315, 147.756114006, 147.914465189, ...   
2  [145.681817532, 145.739325523, 145.895915508, ...   
3  [145.686282158, 145.747886181, 145.766072273, ...   
4  [145.728348255, 145.991557121, 146.004992008, ...   

                                               trial  \
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
1  [1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...   
2  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
3  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
4  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   

                                           trialTime  
0  [-0.0425164699554, 0.030392408371, 0.286720037...  
1  [1.53341960907, 1.63930130005, 1.79765248299, ...  
2  [-0.434995174408, -0.377487182617, -0.22089719...  
3  [-0.430530548096, -0.368926525116, -0.35074043...  
4  [-0.388464450836, -0.125255584717, -0.11182069...

In [5]:
behavior.head()

one       onset      offset  zero  duration   sortIdx  sortLabel
0  1.0  146.116813  148.409139   0.0  2.292326  0.117555       37.0
1  1.0  183.704945  184.279982   0.0  0.575037  0.132874      345.0
2  1.0  212.849418  213.241119   0.0  0.391701  0.134062      635.0
3  1.0  223.417917  223.893709   0.0  0.475791  0.138527      477.0
4  1.0  229.650637  230.018089   0.0  0.367452  0.152330      334.0

### Now separate the activations corresponding to each trial
We will use only time relative to trial here onwards

In [6]:
from spikeHelper.loadSpike import mymax, precisionConvBin
epochs = np.array([[spikes.trialTime[iunit][spikes.trial[iunit]==itrial] for itrial in range(1,behavior.shape[0]+1)] for iunit in range(spikes.shape[0]) ] )
epochs = pd.DataFrame(epochs, index = ['unit'+str(i) for i in range(spikes.shape[0])], columns = ['trial '+str(i) for i in range(1,behavior.shape[0]+1)])

# Make sure spike times are consistent with trial durations
assert ((1000*epochs.apply(lambda x: x.apply(mymax)).max()//1  - np.array([int(1000*behavior.duration[itrial]) for itrial in range(behavior.shape[0])]))>0).sum() == 0

In [7]:
epochs.head()

trial 1  \
unit0  [-0.0425164699554, 0.030392408371, 0.286720037...   
unit1  [1.53341960907, 1.63930130005, 1.79765248299, ...   
unit2  [-0.434995174408, -0.377487182617, -0.22089719...   
unit3  [-0.430530548096, -0.368926525116, -0.35074043...   
unit4  [-0.388464450836, -0.125255584717, -0.11182069...   

                                                 trial 2  \
unit0  [-0.469360589981, -0.299376726151, -0.28987383...   
unit1  [-0.438722610474, -0.42729473114, -0.401244163...   
unit2  [-0.20258808136, -0.171827077866, -0.124395608...   
unit3  [-0.485171079636, -0.39907336235, -0.273612737...   
unit4  [-0.467885971069, -0.355696678162, -0.16289782...   

                                                 trial 3  \
unit0  [-0.409763813019, -0.237117290497, -0.20406270...   
unit1                 [-0.349143028259, -0.133079051971]   
unit2                  [-0.351231813431, 0.338943958282]   
unit3  [-0.435486555099, -0.341401576996, -0.31600642...   
unit4  [-0.454164505005, -0.244694948196, -0.13291501...   

                                                 trial 4  \
unit0  [-0.0172441005707, 0.0825343132019, 0.46157813...   
unit1  [-0.347177028656, -0.300113916397, -0.27136015...   
unit2  [-0.210862159729, -0.159785032272, 0.122347354...   
unit3                 [-0.391495704651, -0.138362884521]   
unit4  [-0.413778066635, 0.0334234237671, 0.121241569...   

                                                 trial 5  \
unit0  [-0.462561368942, -0.457277536392, -0.26214408...   
unit1                [-0.114851951599, -0.0979764461517]   
unit2  [-0.459489345551, -0.377077817917, -0.36634635...   
unit3  [-0.480460882187, -0.266977310181, -0.16076803...   
unit4  [-0.363561153412, -0.246702194214, -0.18108415...   

                                                 trial 6  \
unit0  [-0.412221431732, -0.313794612885, -0.00454664...   
unit1                                                 []   
unit2  [-0.466001987457, -0.154009580612, -0.11722755...   
unit3  [-0.280125379562, -0.170393705368, -0.14499831...   
unit4  [-0.427335739136, -0.316129207611, -0.17489933...   

                                                 trial 7  \
unit0  [-0.367165565491, -0.250019788742, -0.17727494...   
unit1                                   [0.411197423935]   
unit2  [-0.161546230316, -0.10108923912, -0.082739353...   
unit3  [-0.487301111221, -0.348201036453, -0.26734590...   
unit4                                    [0.29630446434]   

                                                 trial 8  \
unit0  [-0.490414142609, -0.23478269577, -0.071311473...   
unit1  [-0.321454048157, 0.378798007965, 0.3975577354...   
unit2  [-0.440729618073, -0.155484199524, -0.14721035...   
unit3  [-0.299540519714, -0.273448944092, -0.07131147...   
unit4  [-0.436347007751, -0.0818791389465, -0.0754072...   

                                                 trial 9  \
unit0                                  [0.0496027469635]   
unit1  [-0.352010250092, -0.254566192627, -0.23101425...   
unit2  [-0.254156589508, 0.0607438087463, 0.122511386...   
unit3                                  [-0.228966236115]   
unit4  [-0.441220998764, -0.262922048569, 0.061563014...   

                                                trial 10  \
unit0  [-0.383590459824, -0.369500160217, -0.22999048...   
unit1  [-0.469606399536, 0.235315084457, 1.0228121280...   
unit2  [-0.34553861618, -0.21962761879, -0.2149581909...   
unit3  [-0.387563705444, -0.228802680969, -0.21016573...   
unit4  [-0.398458957672, -0.383508443832, -0.36950016...   

                             ...                          \
unit0                        ...                           
unit1                        ...                           
unit2                        ...                           
unit3                        ...                           
unit4                        ...                           

                                               trial 792  \
unit0  [-0.3834674

### Now take out unreliable neurons 
Taking into account form, interspike interval and size

In [8]:
import pickle
def filterEpochs(epochs, minFiringRate= 4, minISI=5,method = 'premade', rat = None):
    if method == 'blind':
        goodFR = (epochs.applymap(lambda x: np.sum(x[500:])).mean(axis=1)>minFiringRate).values
        # Dump all neurons that have mean of 1 TOO FAST SPIKE per trial
        goodISI = (epochs.applymap(lambda x: np.histogram(np.diff(np.nonzero(x)),bins = minISI, range = (0,minISI))[0].sum() ).mean(axis=1) < 2).values
        return epochs.iloc[np.logical_and(goodFR,goodISI),:]
    elif method == 'premade':
        assert rat is not None
        goodNeurons = pickle.load(open('Data/goodCellsR'+str(rat)+'_withBaseDiff','rb'))==1
        return epochs.iloc[goodNeurons,:]
    else:
        raise('Nao existe metodo '+ str(method))

In [9]:
print(epochs.shape)
epochs = filterEpochs(epochs, method='premade', rat=rat)
print(epochs.shape)

(63, 801)
(37, 801)


### Bin trials

In [10]:
for itrial in range(1,behavior.shape[0]+1):
    func = lambda x: precisionConvBin(x, int(1000*behavior.duration[itrial-1]), sigma=20, binSize=50)
    epochs['trial '+str(itrial)] = epochs['trial '+str(itrial)].apply(func)

In [40]:
t=np.ceil((.5+ behavior.duration)*(1000/50))==epochs.applymap(len).iloc[0].values

In [25]:
epochs.applymap(len).iloc[:,t.values==False]

trial 13  trial 35  trial 39  trial 46  trial 86  trial 90  trial 104  \
unit0         47        15        37        16        34        47         93   
unit1         47        15        37        16        34        47         93   
unit2         47        15        37        16        34        47         93   
unit3         47        15        37        16        34        47         93   
unit4         47        15        37        16        34        47         93   
unit5         47        15        37        16        34        47         93   
unit6         47        15        37        16        34        47         93   
unit7         47        15        37        16        34        47         93   
unit8         47        15        37        16        34        47         93   
unit9         47        15        37        16        34        47         93   
unit10        47        15        37        16        34        47         93   
unit11        47        15        37        16        34        47         93   
unit12        47        15        37        16        34        47         93   
unit13        47        15        37        16        34        47         93   
unit14        47        15        37        16        34        47         93   
unit15        47        15        37        16        34        47         93   
unit16        47        15        37        16        34        47         93   
unit17        47        15        37        16        34        47         93   
unit18        47        15        37        16        34        47         93   
unit19        47        15        37        16        34        47         93   
unit20        47        15        37        16        34        47         93   
unit21        47        15        37        16        34        47         93   
unit22        47        15        37        16        34        47         93   
unit23        47        15        37        16        34        47         93   
unit24        47        15        37        16        34        47         93   
unit25        47        15        37        16        34        47         93   
unit26        47        15        37        16        34        47         93   
unit27        47        15        37        16        34        47         93   
unit28        47        15        37        16        34        47         93   
unit29        47        15        37        16        34        47         93   
unit30        47        15        37        16        34        47         93   
unit31        47        15        37        16        34        47         93   
unit44        47        15        37        16        34        47         93   
unit51        47        15        37        16        34        47         93   
unit52        47        15        37        16        34        47         93   
unit59        47        15        37        16        34        47         93   
unit62        47        15        37        16        34        47         93   

        trial 231  trial 239  trial 350  trial 385  trial 433  trial 587  \
unit0          49         40         63         30         47         19   
unit1          49         40         63         30         47         19   
unit2          49         40         63         30         47         19   
unit3          49         40         63         30         47         19   
unit4          49         40         63         30         47         19   
unit5          49         40         63         30         47         19   
unit6          49         40         63         30         47         19   
unit7          49         40         63         30         47         19   
unit8          49         40         63         30         47         19   
unit9          49         40         63         30         47         19   
unit10         49         40         63         30         47         19   
unit11         49

---

In [ ]:
filename = 'Data/rato10tudo.mat'
spikes, behavior, epochs = loadSpikeBehav(filename)

# Load all

In [4]:
for rat in [7,8,9,10]:
    #fr=1; isi=5; filename = 'Data/r'+str(rat)+'_filtered_epoch_fr'+str(fr)+'_isi'+str(isi)+'.pickle'
    filename = 'Data/r'+str(rat)+'_filtered_epoch_gabi.pickle'

    if len(glob.glob(filename))==1:
        print('File',filename,'already exists. Loading filtered epochs')
        epochs = pickle.load(open(filename,'rb'))

    else:
        print('File',filename,'does not exist. Filtering epochs and saving file')
        epochs = filterEpochs(epochs,minFiringRate=fr,minISI=isi,method = 'gabi', rat = rat)
        pickle.dump(epochs,open(filename,'wb'))
        
    #trialsOfInterest = (behavior.duration>1).as_matrix() # another option for selecting duration
    duration= epochs.applymap(len).iloc[0].values
    
    ############# SPECIFICS ##############
    epochs = epochs.iloc[:,duration>1300]
    print(epochs.shape)
    subname = '1300msPlus1s'
    binDuration=50
    ############# SPECIFICS ##############
    
    data = XyTfromEpoch(epochs,window=[0,1000],binDuration=binDuration)
    pickle.dump(data,open('Data/'+str(binDuration)+'ms_r'+str(rat)+'_'+subname+'.pickle','wb'))
    print('Done with rat', rat)

File Data/r7_filtered_epoch_gabi.pickle already exists. Loading filtered epochs
(29, 955)
File Data/r8_filtered_epoch_gabi.pickle already exists. Loading filtered epochs
(37, 671)
File Data/r9_filtered_epoch_gabi.pickle already exists. Loading filtered epochs
(29, 683)
File Data/r10_filtered_epoch_gabi.pickle already exists. Loading filtered epochs
(65, 1429)


In [5]:
print('hello world')

hello world


-----------